In [1]:
import MINE.MINE_net as MINE
import tensorflow as tf
import numpy as np
from matplotlib import pyplot as plt

from IPython.display import display, clear_output

I want to make a network and train it to convergence for a range of data samples and regularizations.

In [2]:
def build_stuff(nshape,gradreg,nn = 5,timescale=10,unbias_grad = False,reuse = False,tag = "MINE"):
    
    dat = tf.placeholder(tf.float32,shape = [nshape,nn])
    n1 = tf.placeholder(tf.float32,shape = [nshape,nn])
    n2 = tf.placeholder(tf.float32,shape = [nshape,nn])

    dat1 = dat + n1
    dat2 = dat + n2
    
    mine = MINE.get_MINE(dat1,dat2,grad_timescale = timescale,unbias_grad = unbias_grad,reuse = reuse,tag = tag)

    mi,inp = MINE.get_MINE_grad_vars(dat1,dat2,reuse = True,tag = tag)
    grads = tf.gradients(mi,inp)
    minegradreg = tf.reduce_sum([tf.reduce_mean(g**2,axis = 0) for g in grads])

    loss = -mine + gradreg * minegradreg
    
    return loss, mine,dat,n1,n2
    
    

In [3]:
nn = 10

cov = np.eye(nn)*(1+.1)
ccon = np.eye(nn)*(.1)

MI = (np.log(np.linalg.det(cov)) + np.log(np.linalg.det(ccon)))/2 + nn*(1. + np.log(2*np.pi))

print(MI)

17.3423960981


In [4]:
OUT = []

In [ ]:

for gtime in [.1,1,10,100,1000]:
    tf.reset_default_graph()
    nsam = 100
    ndim = 2
    ntot = 1000
    grad_reg = 0.01

    tag = "{}_{}_{}_{}".format(grad_reg,gtime,ntot,ndim)
    loss,mine,dat,nn1,nn2 = build_stuff(nsam,grad_reg,ndim,gtime,tag = tag,reuse = False,unbias_grad = True)

    alldat = np.random.normal(0,1,[ntot,ndim])
    n1 = np.random.normal(0,.1,[ntot,ndim])
    n2 = np.random.normal(0,.1,[ntot,ndim])

    adam = tf.train.AdamOptimizer(learning_rate = .001)
    train = adam.minimize(loss)

    init = tf.global_variables_initializer()
    sess = tf.Session()
    sess.run(init)

    sess = tf.Session()
    sess.run(init)
    out = []
    for step in range(10000):
        ii = np.random.choice(range(len(alldat)),nsam)
        samp = alldat[ii]
        no1 = n1[ii]
        no2 = n2[ii]

        loss_out,MINEout = sess.run([train,mine],{dat:samp,nn1:no1,nn2:no2})
        if step % 50==0:
            clear_output()
            display("{} {}".format(step,MINEout))

        out.append(MINEout)

    OUT.append([nsam,ndim,ntot,gtime,grad_reg,out])

    print(np.mean(out[-1000:]))
    print(np.std(out[-1000:]))

    plt.plot(range(len(out)),out)
    plt.show()

'9950 2.951181173324585'

3.32013
0.457471


In [ ]:
3.92158
0.883406

def esmooth(x,T = 30):
    t = np.exp(-1./T)
    out = [x[0]]
    for k in x[1:]:
        out.append(out[-1]*t + (1.-t)*k)
    return np.array(out)

In [ ]:
plt.clf()
jj = 0
for o in OUT:
    
    plt.plot(range(len(o[-1])),esmooth(o[-1],T = 1),label = (np.mean(o[-1][-1000:]),np.std(o[-1][-1000:])))
    jj += 1
    
plt.legend()
plt.show()